<a href="https://colab.research.google.com/github/krazyguy/fcc_sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.2 MB/s eta 0:00:00
2.19.0-dev20250110


In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-12 12:36:03--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2025-01-12 12:36:04 (6.21 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-12 12:36:04--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [4]:
train_df=pd.read_csv(train_file_path, delimiter='\t',header=None)

In [5]:
test_df=pd.read_csv(test_file_path, delimiter='\t',header=None)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4179 non-null   object
 1   1       4179 non-null   object
dtypes: object(2)
memory usage: 65.4+ KB


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizer to convert text to integer sequences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")  # Limit to 5000 words, <OOV> for out-of-vocabulary
tokenizer.fit_on_texts(train_df[1])  # Fit the tokenizer on training messages


In [11]:
X_train = tokenizer.texts_to_sequences(train_df[1])
X_test = tokenizer.texts_to_sequences(test_df[1])
max_length = 100
X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

# Display the shape of padded data
print(X_train.shape, X_test.shape)

(4179, 100) (1392, 100)


In [42]:
# Encode labels as 0 (ham) and 1 (spam)
y_train = train_df[0].apply(lambda x: 1 if x == 'spam' else 0).values
y_test = test_df[0].apply(lambda x: 1 if x == 'spam' else 0).values


In [26]:
print(y_train[:10], y_test[:10])


[0 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 0 0 0 0]


In [47]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import Input

# Parameters for transformer model
embedding_dim = 64  # Make sure all dimensions are consistent
num_heads = 4  # Attention heads
num_layers = 2  # Number of transformer layers
units = 64  # Number of units in dense layers

def transformer_model(vocab_size, max_length):
    # Input layer
    inputs = Input(shape=(max_length,))

    # Embedding layer
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)

    # Transformer blocks
    x = embedding
    for _ in range(num_layers):
        # Multi-head Attention layer
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(x, x)
        attention = LayerNormalization()(attention + x)
        x = Dropout(0.1)(attention)

        # Feed-forward network
        ff = Dense(units, activation='relu')(x)
        ff = LayerNormalization()(ff + x)
        x = Dropout(0.1)(ff)

    # Global Average Pooling layer
    x = GlobalAveragePooling1D()(x)

    # Dense output layer
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)

    # Build and compile the model
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the model
model = transformer_model(vocab_size=5000, max_length=100)

# Model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 100, 64)        │        320,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 100, 64)        │         66,368 │ embedding_7[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 100, 64)        │              0 │ multi_head_attention_… │
│                           │                        │                │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_4     │ (None, 100, 64)        │            128 │ add_4[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 100, 64)        │              0 │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 100, 64)        │          4,160 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_5 (Add)               │ (None, 100, 64)        │              0 │ dense_12[0][0],        │
│                           │                        │                │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_5     │ (None, 100, 64)        │            128 │ add_5[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 100, 64)        │              0 │ layer_normalization_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_3    │ (None, 100, 64)        │         66,368 │ dropout_6[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_6 (Add)               │ (None, 100, 64)        │              0 │ multi_head_attention_… │
│                           │                        │                │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_6     │ (None, 100, 64)        │            128 │ add_6[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 100, 64)        │              0 │ layer_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 100, 64)        │          4,160 │ dropout_8[0][0]        │
├──────────────────────

 Total params: 465,793 (1.78 MB)

 Trainable params: 465,793 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Train the model on the training data
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 45s 301ms/step - accuracy: 0.8694 - loss: 0.2789 - val_accuracy: 0.9749 - val_loss: 0.0862
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 39s 290ms/step - accuracy: 0.9911 - loss: 0.0301 - val_accuracy: 0.9835 - val_loss: 0.0511
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 41s 291ms/step - accuracy: 0.9933 - loss: 0.0197 - val_accuracy: 0.9806 - val_loss: 0.0763
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 41s 291ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9871 - val_loss: 0.0975
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 41s 288ms/step - accuracy: 0.9943 - loss: 0.0197 - val_accuracy: 0.9828 - val_loss: 0.0740


In [49]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)

# Print the evaluation results
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


from sklearn.metrics import classification_report

# Predict on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate performance
print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))


44/44 - 4s - 90ms/step - accuracy: 0.9828 - loss: 0.0740
Test Loss: 0.07400196045637131
Test Accuracy: 0.982758641242981
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1205
        spam       0.92      0.95      0.94       187

    accuracy                           0.98      1392
   macro avg       0.96      0.97      0.96      1392
weighted avg       0.98      0.98      0.98      1392



In [50]:
def predict_message(pred_text):
    """
    Predicts if the input message is 'ham' or 'spam' and returns the confidence percentage.

    Args:
        pred_text (str): The input message to classify.

    Returns:
        list: [confidence_percentage, predicted_label]
    """
    # Convert the input text to a sequence of integers using the tokenizer
    pred_text_seq = tokenizer.texts_to_sequences([pred_text])

    # Pad the sequence to the required input length of the model
    pred_text_padded = tf.keras.preprocessing.sequence.pad_sequences(
        pred_text_seq, padding='post', maxlen=max_length
    )

     # Get the prediction from the model
    prediction = model.predict(pred_text_padded, verbose=0)[0][0]

    # Determine the label and the corresponding confidence score
    if prediction > 0.5:
        predicted_label = "spam"
        confidence_score = round(prediction, 4)  # Confidence for spam
    else:
        predicted_label = "ham"
        confidence_score = round(1 - prediction, 4)  # Confidence for ham

    return [confidence_score, predicted_label]
pred_text = "you  won £1000 cash! call to claim your prize."

prediction = predict_message(pred_text)
print(prediction)

[0.9996, 'spam']


In [51]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
